In [ ]:
#Try to compete on https://www.kaggle.com/c/home-data-for-ml-course

# Nov 3: 17877 score
# Nov 4: 14804 score (rank: 2512)

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

In [16]:



# Read the data
X_full = pd.read_csv('train.csv', index_col='Id')

X_test_full = pd.read_csv('test.csv', index_col='Id')


# X_full = shuffle_df(X_full)

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [14]:

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for numerical data
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())
# ])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [15]:
# Define model
model = RandomForestRegressor(n_estimators=500, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])


my_pipeline.fit(X_train,y_train)
preds = my_pipeline.predict(X_valid)

# Evaluate the model using cross-validation
# scores = cross_val_score(my_pipeline, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')

score = mean_absolute_error(y_valid, preds)

# Print the mean absolute error scores
print('MAE:', -score.mean())

/Users/joeng/opt/anaconda3/envs/transformers_env/lib/python3.8/site-packages/sklearn/impute/_base.py:42: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)
/Users/joeng/opt/anaconda3/envs/transformers_env/lib/python3.8/site-packages/sklearn/impute/_base.py:42: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

MAE: -17376.96016438356


In [17]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [18]:
model2= GradientBoostingRegressor(n_estimators=500, random_state=0)

# Preprocessing for numerical data
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())
# ])

my_pipeline2 = Pipeline(steps=[('preprocessor', preprocessor),
                              ('gbm', model2)
                             ])
my_pipeline2.fit(X_train,y_train)
preds2 =my_pipeline2.predict(X_valid)

score = mean_absolute_error(y_valid, preds2)

# Print the mean absolute error scores
print('MAE:', -score.mean())

MAE: -15965.763185108233


In [ ]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': score})
output.to_csv('submission-Xgbm.csv', index=False)

In [14]:
from xgboost import XGBRegressor

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = XGBRegressor(n_estimators=1000, learning_rate=0.03, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

my_pipeline.fit(X_train,y_train)
preds = my_pipeline.predict(X_valid)

score = mean_absolute_error(y_valid, preds)

# Print the mean absolute error scores
print('MAE:', -score.mean())

MAE: -17123.897126498287


In [18]:
preds1_test =my_pipeline.predict(X_test)
preds2_test =my_pipeline2.predict(X_test)

preds_final = (preds1_test + preds2_test) / 2

# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_final})
output.to_csv('submission.csv', index=False)

In [20]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds2_test})
output.to_csv('submission2.csv', index=False)

In [7]:
# try to include the SelectKBest , but not working ...

from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error

# Define the number of features to select
k = 2

# Define the feature selection step
feature_selection = SelectKBest(score_func=f_regression, k=k)

# Define the preprocessing steps for numerical and categorical data
numerical_transformer = SimpleImputer(strategy='median')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the Random Forest regression model
model = RandomForestRegressor(n_estimators=500, random_state=0)

# Bundle preprocessing, feature selection, and modeling code in a pipeline
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selection', feature_selection),
    ('model', model)
])

# Fit the pipeline to the training data
my_pipeline.fit(X_train, y_train)

# Get the selected feature names
# selected_features = X_train.columns[feature_selection.get_support()]

# Predict the target variable using the trained model
y_pred = my_pipeline.predict(X_valid)

# Evaluate the model using mean absolute error
score = mean_absolute_error(y_valid, y_pred)

# Print the selected feature names and the mean absolute error score
# print('Selected features:', list(selected_features))
print('MAE:', score)

MAE: 27305.408108624597


In [4]:
# try to include the Stratify continuous methods to the y-value

def stratify_continuous(n_folds, y):
    """Stratify a dataset on a continuous target."""
    if n_folds < 2 or n_folds > 10:
        raise ValueError("Please select a number of folds from 2 to 10.")
    
    fold_nums = list(range(n_folds))

    # DataFrame where "index" column contains the original indices
    df = pd.DataFrame(
        y
        # Shuffle before ranking so that cars with the same price are ordered randomly.
        .sample(frac = 1, random_state = 1, ignore_index = False)
    )

    # This column gives a rank to each value in y. 0 is the rank of the lowest value.
    # Ties are broken according to order of appearance.
    df["rank"] = df[y.name].rank(method = "first") - 1

    df["fold"] = 0

    for f in fold_nums[1:]:
        # start at f, then increment by n_folds
        indices = list(range(f, df.shape[0], n_folds))
        df.loc[df["rank"].isin(indices), "fold"] = f

    # Revert df to original order of indices
    df = df.reindex(index = y.index)

    # A series that indicates the fold number of each observation according to its original position in y
    fold_series = df["fold"].copy()

    return fold_series



In [12]:


def split_folds(X, y, fold_series, test_fold):
    """Take a dataset whose observations have been grouped into folds,
    then perform a train-test split."""

    if fold_series.dtype != "int64":
        raise AttributeError("The fold list does not purely contain integers.")

    test_mask = (fold_series == test_fold)

    X_train = X.loc[~test_mask].copy()
    y_train = y.loc[~test_mask].copy()

    X_test = X.loc[test_mask].copy()
    y_test = y.loc[test_mask].copy()

    return X_train, X_test, y_train, y_test



In [7]:
# Read the data
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

In [11]:
# Access the ColumnTransformer object
preprocessor = my_pipeline2.named_steps['preprocessor']

# Access the categorical transformer and get the names of the one-hot encoded features
categorical_transformer = preprocessor.named_transformers_['cat']
one_hot_encoder = categorical_transformer.named_steps['onehot']
one_hot_encoded_feature_names = one_hot_encoder.get_feature_names(categorical_cols)

# Access the numerical transformer and get the names of the numerical features
numerical_transformer = preprocessor.named_transformers_['num']
numerical_feature_names = numerical_cols

# Print the names of the categorical and numerical features
print('Categorical feature names:', list(one_hot_encoded_feature_names))
print('Numerical feature names:', numerical_feature_names)

Categorical feature names: ['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'Alley_Grvl', 'Alley_Pave', 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_AllPub', 'Utilities_NoSeWa', 'LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Gtl', 'LandSlope_Mod', 'LandSlope_Sev', 'Condition1_Artery', 'Condition1_Feedr', 'Condition1_Norm', 'Condition1_PosA', 'Condition1_PosN', 'Condition1_RRAe', 'Condition1_RRAn', 'Condition1_RRNe', 'Condition1_RRNn', 'Condition2_Artery', 'Condition2_Feedr', 'Condition2_Norm', 'Condition2_PosA', 'Condition2_PosN', 'Condition2_RRAe', 'BldgType_1Fam', 'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE', 'HouseStyle_1.5Fin', 'HouseStyle_1.5Unf', 'HouseStyle_1Story', 'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'HouseStyle_2Story',